# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI


In [2]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"
    

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/08/06/outsmart/',
 'https://edwarddonner.com/2024/08/06/outsmart/',
 'https://edwarddonner.com/2024/06/26/choosing-the-right-llm-resources/

In [6]:
udn = Website("https://udn.com/news/index")
udn.links

['/news/index',
 '/news/index',
 'https://vip.udn.com/vip/index?utm_source=udn&utm_medium=referral&utm_campaign=familybar',
 'https://money.udn.com/money/index?utm_source=udn&utm_medium=referral&utm_campaign=familybar',
 'https://sdgs.udn.com/?utm_source=udn&utm_medium=referral&utm_campaign=familybar',
 'https://udesign.udnfunlife.com/mall/Cc1a00.do?sid=91_udn&utm_source=udn.com&utm_medium=referral_familybar_udn&utm_term=familybar_udn&utm_campaign=20170118_UDN',
 'https://utravel.udn.com/index.jsp?utm_source=udn&utm_medium=referral&utm_campaign=familybar',
 'https://visionproject.org.tw/?utm_source=udn&utm_medium=referral&utm_campaign=familybar',
 'https://udn.com/mypage/mynewsFeature?utm_source=udn&utm_medium=referral&utm_campaign=familybar',
 'https://vip.udn.com/',
 '//member.udn.com/member/center.jsp?hold=y',
 '//member.udn.com/member/ShowMember?actiontype=update',
 '//member.udn.com/member/order_list.jsp',
 '//member.udn.com/member/ProcessLogout',
 '/news/index',
 '/news/breaknews

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [7]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [9]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [10]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/
https://edwarddonner

In [11]:
print(get_links_user_prompt(udn))

Here is the list of links on the website of https://udn.com/news/index - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/news/index
/news/index
https://vip.udn.com/vip/index?utm_source=udn&utm_medium=referral&utm_campaign=familybar
https://money.udn.com/money/index?utm_source=udn&utm_medium=referral&utm_campaign=familybar
https://sdgs.udn.com/?utm_source=udn&utm_medium=referral&utm_campaign=familybar
https://udesign.udnfunlife.com/mall/Cc1a00.do?sid=91_udn&utm_source=udn.com&utm_medium=referral_familybar_udn&utm_term=familybar_udn&utm_campaign=20170118_UDN
https://utravel.udn.com/index.jsp?utm_source=udn&utm_medium=referral&utm_campaign=familybar
https://visionproject.org.tw/?utm_source=udn&utm_medium=referral&utm_campaign=familybar
https://udn.com/mypage/mynewsFeature?utm_source=udn&utm_medium=referral&utm_c

In [12]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)


In [13]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/meta-llama/Llama-3.3-70B-Instruct',
 '/Datou1111/shou_xin',
 '/tencent/HunyuanVideo',
 '/black-forest-labs/FLUX.1-dev',
 '/deepseek-ai/DeepSeek-V2.5-1210',
 '/models',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/multimodalart/flux-style-shaping',
 '/spaces/Kwai-Kolors/Kolors-Virtual-Try-On',
 '/spaces/ginipick/FLUXllama',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces',
 '/datasets/HuggingFaceFW/fineweb-2',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/CohereForAI/Global-MMLU',
 '/datasets/O1-OPEN/OpenO1-SFT',
 '/datasets/aiqtech/kolaw',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/diffuse

In [14]:
get_links("https://huggingface.co")

{'links': [{'type': 'home page', 'url': 'https://huggingface.co/'},
  {'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

In [15]:
get_links("https://udn.com/news/index")

{'links': [{'type': 'about page', 'url': 'https://www.udngroup.com/'},
  {'type': 'contact us page', 'url': 'https://co.udn.com/co/contactus'},
  {'type': 'careers page', 'url': 'https://member.udn.com/member/rule.jsp'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [16]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [17]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Trending on
this week
Models
meta-llama/Llama-3.3-70B-Instruct
Updated
5 days ago
•
147k
•
1.01k
Datou1111/shou_xin
Updated
6 days ago
•
15.3k
•
391
tencent/HunyuanVideo
Updated
8 days ago
•
4.39k
•
1.03k
black-forest-labs/FLUX.1-dev
Updated
Aug 16
•
1.36M
•
7.27k
deepseek-ai/DeepSeek-V2.5-1210
Updated
4 days ago
•
2.02k
•
175
Browse 400k+ models
Spaces
Running
on
Zero
1.27k
🏢
TRELLIS
Scalable and Versatile 3D Generation from images
Running
on
L40

In [18]:
print(get_all_details("https://udn.com/news/index"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.udngroup.com/'}, {'type': 'company page', 'url': 'https://udn.com/'}, {'type': 'careers page', 'url': 'https://member.udn.com/member/center.jsp'}, {'type': 'contact us page', 'url': 'https://co.udn.com/co/contactus'}]}
Landing page:
Webpage Title:
聯合新聞網：觸動未來新識力
Webpage Contents:
聯合報
經濟日報
陽光行動
有設計
有行旅
願景
我的新聞
會員中心
我的設定
訂閱管理
登出
即時
要聞
娛樂
運動
全球
社會
地方
產經
股市
房市
生活
寵物
健康
橘世代
文教
評論
兩岸
科技
Oops
閱讀
旅遊
雜誌
報時光
倡議+
500輯
轉角國際
NBA
時尚
汽車
棒球
HBL
遊戲
專題
網誌
女子漾
購物
全產品速覽
新聞評論
聯合新聞網
陽光行動
聯合報數位版
聯合70
轉角國際
倡議家
世界日報
新聞話題
服務
會員中心
U利點數
我的新聞
歷史新聞
活動專區
udn粉絲團
udn line 好友
股市理財
經濟日報網
房地產
中經社
樂透
發票
生活娛樂
噓！星聞
優人物
udnSTYLE
500輯
遊戲角落
發燒車訊
元氣網
橘世代
寵物部落
科技玩家
女子漾
報時光
運動
udn NBA
野球夢田
HBL
閱讀創作
琅琅悅讀
琅琅書店
讀創故事
一刻鯨選
聯副70
udn部落格
聯合文學
聯經出版
聯文雜誌
文創購物
數位文創
IP 授權
瘋活動
售票網
時光商號
更多產品
聯合知識庫
聯合電子報
聯合影音網
聯合學苑
有行旅
APP行動網
內容聯盟
關於我們
P
訂閱
刊登廣告
FAQ
·
客服
新聞授權
服務條款
·
著作權
·
隱私權聲明
聯合報系
訂報紙
關於我們
網站總覽
聯合線上公司 著作權所有 ©2023
我的頻道
* 拖拉選單自訂排序儲存
恢復預設
確定
設定
加入橘世代VIP富活人生下半場。
即時
要聞
快訊
不願同流合汙...大

In [29]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [21]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [22]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\nmeta-llama/Llama-3.3-70B-Instruct\nUpdated\n5 days ago\n•\n147k\n•\n1.01k\nDatou1111/shou_xin\nUpdated\n6 days ago\n•\n15.3k\n•\n392\ntencent/HunyuanVideo\nUpdated\n8 days ago\n•\n4.39k\n•\n1.03k\nblack-forest-labs/FLUX.1-dev\nUpdated\nAug 16\n•\n1.36M\n•\n7.27k\ndeepseek-ai/DeepSeek-V2.5-1210\nUpdated\n4 days ago\n•\n2.02k\n•\n175\nBrowse 400k+ models\nSpaces\nRunning\non\nZero\n1.28k\n🏢\nTRELLIS\nScalable and Versatile 3D Generati

In [23]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [24]:
create_brochure("HuggingFace", "https://huggingface.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.com'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.com/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.com/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.com/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

---

## Welcome to Hugging Face  
**The AI community building the future.**  
Join a vibrant platform where the machine learning community collaborates on models, datasets, and applications.

### About Us
Hugging Face is dedicated to shaping the future of artificial intelligence by providing a collaborative space for developers, researchers, and enterprises to create, discover, and improve machine learning models and datasets. With over 400,000 models and 100,000 datasets available, our platform serves as the heart of the AI community.

---

### Our Offerings

- **Models**: Explore the latest innovations in ML with our extensive library, featuring trending models such as *Llama-3.3* and *HunyuanVideo*.

- **Datasets**: Access a wide array of datasets essential for various machine learning tasks, including NLP, computer vision, and audio processing.

- **Spaces**: Collaborate and showcase applications in real-time, making it easier to experiment and build new solutions together.

- **Enterprise Solutions**: Tailored services offering advanced security, dedicated support, and resource management to empower organizations in deploying AI technologies. Pricing starts at **$20/user/month**.

- **Community Support and Open Source**: We prioritize community contributions and collaborative development, with our open-source projects like *Transformers* and *Diffusers* garnering massive user engagement.

---

### Company Culture
At Hugging Face, we believe in fostering a community-driven environment that encourages innovation, collaboration, and the sharing of knowledge. We strive to empower individuals from all backgrounds to contribute to the evolving landscape of machine learning. Our focus on inclusivity enables us to tap into diverse perspectives, leading to groundbreaking advancements in AI.

---

### Who We Serve
More than **50,000 organizations** trust Hugging Face, including renowned names such as:
- **Google**
- **Microsoft**
- **Amazon Web Services**
- **Meta**

These partnerships fuel our mission to enhance AI capabilities across various industries, from small startups to large enterprises.

---

### Careers at Hugging Face
Join our team of passionate individuals who are passionate about artificial intelligence. We're on the lookout for talented engineers, researchers, and creators who want to make an impact in the AI space. You can explore our available job opportunities on our **[Careers Page](#)**.

---

### Get Involved
Ready to explore the possibilities? Join the Hugging Face community today!

- **[Sign Up](#)** for free.
- Follow us on social media to stay up-to-date with the latest trends and community discussions.
    - [GitHub](#)
    - [Twitter](#)
    - [LinkedIn](#)
    - [Discord](#)

**Hugging Face – The AI community building the future.**  
Together, we can build a better tomorrow through AI.

--- 

*For more information, visit our website at [huggingface.co](#).*

In [25]:
create_brochure("Microsoft", "https://www.microsoft.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.microsoft.com/en-us/about'}, {'type': 'careers page', 'url': 'https://careers.microsoft.com/'}, {'type': 'company page', 'url': 'https://www.microsoft.com/en-us/investor/default.aspx'}, {'type': 'diversity page', 'url': 'https://www.microsoft.com/en-us/diversity/'}, {'type': 'accessibility page', 'url': 'https://www.microsoft.com/en-us/accessibility/'}, {'type': 'sustainability page', 'url': 'https://www.microsoft.com/en-us/sustainability/'}]}


# Microsoft Company Brochure

## Overview
**Microsoft** is a global technology company that empowers every person and organization on the planet to achieve more. With a vast array of products and services, including software solutions, hardware, gaming, and cloud computing, Microsoft is at the forefront of innovation in the tech industry.

## Core Offerings
- **Microsoft 365**: A subscription-based service that provides access to Office applications and other productivity services enabled over the internet.
- **Windows**: The leading operating system that facilitates user-friendly interfaces for personal and professional computing.
- **Surface Devices**: A line of touchscreen personal computers and interactive whiteboards that blend portability with performance.
- **Xbox**: A gaming console that offers immersive gaming experiences through access to games, subscriptions, and online services.

## Customer Focus
Microsoft serves a diverse clientele ranging from individual consumers to large enterprises. Key areas include:
- **Small Businesses**: Tailored tools and services designed to help small businesses thrive.
- **Enterprises**: Comprehensive solutions like **Microsoft Azure** and **Dynamics 365** that drive business intelligence and cloud computing.
- **Gamers**: Engaging products, services, and communities around the Xbox ecosystem, including Game Pass and Xbox Live.

## Company Culture
At Microsoft, company culture is centered around innovation, collaboration, and inclusion. The company fosters an environment where creativity thrives and diverse perspectives are celebrated. Core values include:
- **Growth Mindset**: Encouraging continuous learning and improvement.
- **Inclusivity**: Valuing diversity in backgrounds and opinions to drive better decision-making and innovation.
- **Customer Obsession**: Commitment to delivering exceptional experiences to all customers.

## Careers at Microsoft
Microsoft is dedicated to attracting and retaining top talent. They offer a wide range of career opportunities across various fields, including technology, sales, marketing, and more. Employees are encouraged to:
- **Pursue professional development**: Ongoing training and learning opportunities.
- **Engage with communities**: Participate in community outreach and service projects.
- **Innovate**: Work on groundbreaking projects that shape the future of technology.

## Join Us
Whether you are a customer looking for industry-leading solutions, an investor seeking opportunities in a tech giant, or a prospective employee aiming to work in an innovative environment, Microsoft welcomes you to be a part of their mission to empower every person and organization on the planet.

For more information or to explore career opportunities, visit [Microsoft Careers](https://careers.microsoft.com/).

---

This brochure reflects Microsoft’s commitment to excellence and opportunity in technology and business.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [26]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)


In [27]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## Welcome to Hugging Face
**The AI community building the future.**  
Hugging Face is a leading platform that fosters collaboration within the machine learning (ML) community. Here, you can discover, create, and share models, datasets, and applications effectively.

---

## Our Offerings

- **Models:** Access over **400k models** ranging from cutting-edge natural language processing to image generation.
  
- **Datasets:** Browse and utilize a vast pool of **100k datasets** to empower your ML projects.
  
- **Spaces:** Explore applications hosted in the community, featuring interactive demos and in-development projects.
  
- **Enterprise Solutions:** Tailored for teams needing advanced capabilities, including enterprise-grade security and dedicated support.

---

## Who We Serve
With over **50,000 organizations** leveraging Hugging Face to enhance their machine learning capabilities, our diverse clientele includes:

- **Big Tech Giants:** Amazon Web Services, Google, Microsoft, Meta
- **Innovative Startups:** Ai2, Grammarly, Writer 
- **Educational Institutions:** Various academic collaborations

Whether you’re an enterprise, startup, or researcher, our community-driven platform has something unique to offer.

---

## Company Culture
At Hugging Face, culture is defined by **collaboration, innovation, and openness.** We embrace a community-first approach and prioritize transparency, welcoming contributions from anyone passionate about machine learning.

- **Open Source:** Our commitment to open-source principles is foundational, with resources like Transformers, Diffusers, and Tokenizers empowering developers globally.
  
- **Continuous Learning:** Employees and community members are encouraged to share knowledge and grow through forums, documentation, and events.

- **Flexibility and Inclusivity:** We prioritize flexibility in work environment and availability to ensure every voice is heard and valued.

---

## Careers at Hugging Face
Join a passionate team of AI enthusiasts who are dedicated to building the future of machine learning. We’re always looking for talented individuals in various domains including:

- Machine Learning
- Data Science
- Software Development
- Community Management

At Hugging Face, you’ll have the opportunity to work on the cutting edge of AI and make a tangible impact in the tech community.

---

## Get Involved
Whether you're a prospective customer, a potential investor, or someone looking to join our team, we invite you to explore the world of Hugging Face. 

**Sign up** for a free account or explore our offerings today at [Hugging Face](https://huggingface.co).

---

For any inquiries, follow us on our social channels or contact us directly. Together, let's build the future of AI!

In [30]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://huggingface.co/meta-llama'}]}



# 🤗 Welcome to Hugging Face! 🤗

### The AI Community Building the Future (with at least 50,000 Friends)

---

**Who Are We?**  
At Hugging Face, we’re more than just a bunch of nerdy self-proclaimed wizards in the world of Machine Learning—we’re a thriving community that loves collaboration like kids love candy! We build models, datasets, and applications that are the AI equivalent of Swiss Army knives—versatile, useful, and just sharp enough to make your work tidy.

---

## 🛠️ Our Toolbox
- **Models**: Forget shoveling sand; we’ve got over **400k+ models** for every niche, need, and perhaps an unfulfilled self-dream of yours! Build, share, and shout from the rooftops about *HuMan* - the first robot to do your laundry!
  
- **Datasets**: With **100k+ datasets** at your disposal, no longer must your **data woes** keep you up at night! We offer datasets for everything, even that pizza preference research you always wanted to conduct.

- **Spaces**: Create and collaborate in **spaces** that make a studio apartment in New York look spacious! Think of it as your ML sanctuary where ideas flow like wine at a Greek wedding.

---

## 🚀 For Our Silly Customers (And Serious Ones Too!)
Join over **50,000 organizations** like Microsoft, Meta, Google, and even a tiny non-profit called Ai2! They enjoy our enterprise solutions—because who wouldn’t want to step into the future with a little help from their AI friends? 

Do you want to optimize your business or just prank your office mate's printer? We can help!

---

## 🌟 Join Our Team (Or at least check if we really wear capes)
### Come For The Models, Stay For The Culture!
- **Work, but make it fun**: Collaboration is our middle name (okay, it's not...but it should be).
- **Flexible Hours**: Sometimes your best ideas strike at 3 AM. You do you!
- **Free Coffee**: It’s like a hug in a mug! Plus, what’s better than caffeine-fueled brainstorming? 
- **Incredible Colleagues**: We’re a colorful mix of experts ensuring there's never a dull moment. And we have the sweetest AI community in our corner!

---

## 🚪 Careers
Want to join the fun? Check out our **current openings**! We're looking for tech-savvy, caffeinated dreamers who can juggle datasets while coding with one hand and enjoying a slice of pizza with the other.

---

## 📢 Be Part of the Future!
So why wait? Sign up, log in, or tango in your living room—it all starts here at Hugging Face! Because at the end of the day, we believe in building a world not just powered by AI, but *with* it. 

---

**Hugging Face—where the future of AI gets a group hug!** 🤗

For more info ask our friendly bots or visit **[huggingface.co](https://huggingface.co)**. 
 

Feel free to adjust any humorous aspects to better align with the brand's personality or tone!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>